## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-25-15-52-50 +0000,bbc,Nollywood actor Odira Nwobu dies in South Afri...,https://www.bbc.com/news/articles/c7734d46g45o...
1,2025-11-25-15-52-35 +0000,wapo,U.S. and Ukraine finalizing peace deal but Rus...,https://www.washingtonpost.com/world/2025/11/2...
2,2025-11-25-15-49-31 +0000,bbc,Jury trials could be scrapped except in most s...,https://www.bbc.com/news/articles/cy7vdvrnnvzo...
3,2025-11-25-15-47-38 +0000,bbc,Everything you need to know about the Budget,https://www.bbc.com/news/articles/cj4w44w42j5o...
4,2025-11-25-15-46-00 +0000,nyt,Trump Administration Is Taking Billions in Sta...,https://www.nytimes.com/2025/11/25/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2340/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
27,trump,45
8,ukraine,26
10,peace,25
66,new,20
222,plan,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
296,2025-11-24-20-17-58 +0000,nypost,"US, Ukraine agree on new 19-point peace plan t...",https://nypost.com/2025/11/24/world-news/us-uk...,150
68,2025-11-25-12-56-44 +0000,nypost,"Trump Army secretary meets Russians after US, ...",https://nypost.com/2025/11/25/us-news/trump-ar...,146
179,2025-11-25-03-24-31 +0000,nypost,Initial Ukraine-Russia peace plan was hashed o...,https://nypost.com/2025/11/24/us-news/initial-...,130
207,2025-11-25-00-42-05 +0000,nyt,What to Know About Trump’s Peace Plan for Russ...,https://www.nytimes.com/2025/11/24/us/politics...,128
247,2025-11-24-22-43-00 +0000,wsj,Ukraine Peace-Plan Negotiators Leave Thorniest...,https://www.wsj.com/world/europe/big-gaps-stil...,122


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
296,150,2025-11-24-20-17-58 +0000,nypost,"US, Ukraine agree on new 19-point peace plan t...",https://nypost.com/2025/11/24/world-news/us-uk...
319,62,2025-11-24-18-33-00 +0000,wsj,President Trump’s advisers are discussing a pr...,https://www.wsj.com/politics/policy/trump-advi...
179,60,2025-11-25-03-24-31 +0000,nypost,Initial Ukraine-Russia peace plan was hashed o...,https://nypost.com/2025/11/24/us-news/initial-...
64,47,2025-11-25-13-02-00 +0000,wsj,Speaker Mike Johnson cautioned the White House...,https://www.wsj.com/politics/policy/speaker-jo...
313,46,2025-11-24-18-57-50 +0000,cbc,"Criminal cases against ex-FBI director Comey, ...",https://www.cbc.ca/news/world/us-va-case-dismi...
32,45,2025-11-25-14-38-57 +0000,nypost,Depraved sicko admits he murdered estranged wi...,https://nypost.com/2025/11/25/us-news/man-admi...
194,42,2025-11-25-02-00-00 +0000,wsj,Allies of Federal Reserve Chair Jerome Powell ...,https://www.wsj.com/economy/central-banking/fe...
47,35,2025-11-25-13-59-45 +0000,nypost,Video shows moment ‘Slender Man’ stabber makes...,https://nypost.com/2025/11/25/us-news/video-sh...
272,30,2025-11-24-21-30-10 +0000,nypost,DOJ settles case accusing real estate tech fir...,https://nypost.com/2025/11/24/business/doj-set...
8,30,2025-11-25-15-38-35 +0000,nypost,Treasury Secretary Scott Bessent says there’s ...,https://nypost.com/2025/11/25/business/bessent...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
